In [1]:
import nltk
#nltk.download()
from __future__ import unicode_literals
from hazm import *
import re

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#importing data from file
df=pd.read_excel('failure_description.xlsx')

In [4]:
#looking to dataframe
df.iloc[:,1][0]

'بازوهای بیرونی شماره21و6 گریسکاری نمیشود'

In [5]:
#func for removing english character
def remove_eng(x):
    return re.sub('[0-9a-zA-Z]',' ',x)

In [6]:
#mapping function to dataframe
df['Problem_Description'] = df.iloc[:,1].map(lambda x: remove_eng(x))

In [7]:
#looking
df

,Cause_of_failure,Problem_Description
0,NaN,بازوهای بیرونی شماره و گریسکاری نمیشود
1,NaN,نازل های اب روی تجهیز گرفته است
2,NaN,تجهیز الارم دارد
3,NaN,دستگاه گلس بید یکی از عملکرد های ان در هنگام ک...
4,NaN,دستگاه گلاس بید استارت نمیشود.
...,...,...
59388,NaN,بالا رفتن دمای یاتاقان پنیون بال میل.
59389,NaN,نیاز به نصب داربست در اطراف بدنه بال میل.
59390,سایش,شل بودن وی بلت
59391,NaN,درخواست ساخت و تعویض داکت های ماقبل و بعد از ن...


In [8]:
#useing HAZM for normalizing sentenses
normalizer = Normalizer(token_based=True)
def norm(x):
    return normalizer.normalize(x)

In [9]:
#sample test 
norm(df.iloc[:,1][0])

'بازوهای بیرونی شماره و گریسکاری نمیشود'

In [10]:
#mapping function to dataframe
df['Normalized_description'] = df.iloc[:,1].map(lambda x: norm(x))

In [11]:
#useing HAZM for tokenizing
def word_tok(x):
    return word_tokenize(x)
df['word_tokenize'] = df.iloc[:,2].map(lambda x: word_tok(x))

In [12]:
#looking
df.iloc[:,3][0]

['بازوهای', 'بیرونی', 'شماره', 'و', 'گریسکاری', 'نمیشود']

In [13]:
#removing empty item and space from begin and end of words
def remove_space(x):
    remove_space = [item.strip() for item in x]
    delete_empty = [item for item in remove_space if item.strip()]
    return delete_empty
    
df['removed_space'] = df.iloc[:,3].map(lambda x: remove_space(x))        

In [14]:
#sample test
a=[' یتبنیمشس سینش  ','','نسشیت    ']
print(a)
remove_space(a)

[' یتبنیمشس سینش  ', '', 'نسشیت    ']


['یتبنیمشس سینش', 'نسشیت']

In [15]:
#removing stopwords
stop_words_df=pd.read_excel('stop_words.xlsx')
stop_words = stop_words_df.iloc[:,0].to_list()
def remove_stopwords(x):
    output=[]
    for item in x:
        if item not in stop_words:
            output.append(item)
    return output

In [16]:
#mapping function to dataframe
df['removed_stopwords'] = df.iloc[:,4].map(lambda x: remove_stopwords(x))

In [17]:
#making sentence by joining
df['sentens'] = df.iloc[:,5].map(lambda x: ' '.join(x))

In [18]:
#making new dataframe by selecting some colume from old dataframe
temp=df[['Cause_of_failure','sentens','removed_stopwords']].copy()

In [19]:
#removing NaN
final_df=temp[temp['Cause_of_failure'].notnull()]

In [20]:
#looking
final_df

,Cause_of_failure,sentens,removed_stopwords
5,شکستگی,شفتهای برگشتی ستون شکسته_است,"[شفتهای, برگشتی, ستون, شکسته_است]"
13,عدم تنظیم,تعمیر جنوب انجام,"[تعمیر, جنوب, انجام]"
19,انحراف,رفع انحراف نوار انجام,"[رفع, انحراف, نوار, انجام]"
24,عیب هیدرولیکی,الارم تجهیز دلیل کاهش فشار پمپ گریس اتفاق افتا...,"[الارم, تجهیز, دلیل, کاهش, فشار, پمپ, گریس, ات..."
33,شکستگی,پایپهای جنوب چکش ثابت افتاده_است,"[پایپهای, جنوب, چکش, ثابت, افتاده_است]"
...,...,...,...
59361,سایش,گپ رولهای شمارش پایین تنظیم خارج‌شده گشاد تنظی...,"[گپ, رولهای, شمارش, پایین, تنظیم, خارج‌شده, گش..."
59362,سایش,گپ رولهای شمارش پایین تنظیم خارج‌شده گشاد تنظی...,"[گپ, رولهای, شمارش, پایین, تنظیم, خارج‌شده, گش..."
59363,سایش,گپ رولهای شمارش پایین تنظیم خارج‌شده گشاد تنظی...,"[گپ, رولهای, شمارش, پایین, تنظیم, خارج‌شده, گش..."
59364,سایش,گپ رولهای شمارش پایین تنظیم خارج‌شده گشاد تنظی...,"[گپ, رولهای, شمارش, پایین, تنظیم, خارج‌شده, گش..."


In [21]:
#changing categure value by 0,1,2,....
res = dict(zip(final_df.Cause_of_failure.value_counts().keys().to_list(),range(27)))
final_df.Cause_of_failure.replace(res,inplace = True)

C:\Users\h.alavi\AppData\Local\Temp\ipykernel_6396\3887569738.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.Cause_of_failure.replace(res,inplace = True)


In [22]:
#changed by this values
res.items()

dict_items([('سایش', 0), ('عدم عملکرد صحیح', 1), ('شل بودن', 2), ('عدم تنظیم', 3), ('شکستگی', 4), ('انحراف', 5), ('افزایش ویبره', 6), ('پایان عمر مفید', 7), ('صدای غیر عادی', 8), ('پارگی', 9), ('دفرمگی', 10), ('نشتی روغن', 11), ('الودگی زیاد', 12), ('گرفتگی', 13), ('سوختگی', 14), ('نشتی اب', 15), ('ترک خوردگی', 16), ('افزایش دما', 17), ('نشتی هوا', 18), ('زنگ زدگی', 19), ('خرابی نسوز و عایق', 20), ('فشار غیرعادی', 21), ('نشتی گاز', 22), ('کمبود روغن', 23), ('عیب هیدرولیکی', 24), ('ترکیدگی', 25)])

In [23]:
#reindexing dataframe
final_df.reset_index(inplace= True,drop=True)

In [24]:
#looking
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6594 entries, 0 to 6593
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Cause_of_failure   6594 non-null   int64 
 1   sentens            6594 non-null   object
 2   removed_stopwords  6594 non-null   object
dtypes: int64(1), object(2)
memory usage: 154.7+ KB


In [25]:
#save cleaned dataframe to excel file
#final_df.to_excel('f_data.xlsx')

In [26]:
#Vectorizing data by sklearn "Bag Of Words"
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [27]:
#appling model to dataframe
bow = cv.fit_transform(final_df.iloc[:,1]).toarray()

In [28]:
bow.shape

(6594, 2684)

In [29]:
#Vectorizing data by sklearn "TF-idf"
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
#making class
tf = TfidfVectorizer()

In [31]:
#training data
tf.fit(final_df.iloc[:,1])

TfidfVectorizer()

In [32]:
#looking to vocabulary
print(tf.fit(final_df.iloc[:,1]).vocabulary_)

{'شفتهای': 1354, 'برگشتی': 455, 'ستون': 1207, 'شکسته_است': 1384, 'تعمیر': 625, 'جنوب': 735, 'انجام': 253, 'رفع': 1036, 'انحراف': 255, 'نوار': 1977, 'الارم': 208, 'تجهیز': 564, 'دلیل': 928, 'کاهش': 2467, 'فشار': 1511, 'پمپ': 2353, 'گریس': 2611, 'اتفاق': 24, 'افتاده_است': 195, 'پایپهای': 2311, 'چکش': 2428, 'ثابت': 695, 'نصب': 1943, 'اسکافلد': 146, 'جهت': 740, 'پرژکتور': 2336, 'های': 2015, 'جلو': 725, 'پولپیت': 2375, 'بالینگ': 392, 'هماهنگ': 2039, 'اقای': 203, 'ثقفی': 696, 'گیربکس': 2649, 'روغن': 1059, 'پیچهای': 2400, 'النی': 219, 'بازوی': 364, 'بالایی': 388, 'امده': 233, 'نشتی': 1936, 'اب': 1, 'سقف': 1239, 'لوله': 1646, 'فاضلاب': 1490, 'پست': 2338, 'لطفا': 1627, 'بررسی': 432, 'عمرانی': 1465, 'شل': 1356, 'شدگی': 1331, 'پیچ': 2395, 'براکت': 416, 'ساپورت': 1185, 'ویل': 2276, 'بیرونی': 535, 'غرب': 1483, 'مشاهده': 1754, 'گردید': 2593, 'دابل': 829, 'فلاپ': 1521, 'شمال': 1362, 'درام': 863, 'استراکچر': 106, 'همراه': 2044, 'دیگه': 989, 'تجهیزات': 565, 'میل': 1872, 'بلت': 491, 'خرابی': 795, 'ریترن

In [33]:
#making vectors by TF-idf
X=tf.transform(final_df.iloc[:,1]).toarray()

In [34]:
y=final_df['Cause_of_failure']

In [35]:
X.shape

(6594, 2684)

In [36]:
#spliting data to train and test dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=123)

In [37]:
X_train.shape

(5275, 2684)

In [38]:
#training model 
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB().fit(X_train,y_train)

In [39]:
#predicting 
y_pred=model.predict(X_test)

In [40]:
#calculating accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.5678544351781653

In [41]:
#calculating accuracy
y_predict=model.predict(X_train)
accuracy_score(y_predict,y_train)

0.5992417061611375

In [43]:
#Indexing words of corpus by tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

In [53]:
#Indexing
max_words = 500
tkn = Tokenizer(num_words=max_words)
tkn.fit_on_texts(final_df.iloc[:,1])
seq = tkn.texts_to_sequences(final_df.iloc[:,1])
print(len(seq))
for i in range(10):
    print(seq[i],'<----->',final_df.iloc[:,1][i])

6594
[423, 302, 69, 12] <-----> شفتهای برگشتی ستون شکسته_است
[128, 2, 15] <-----> تعمیر جنوب انجام
[34, 16, 43, 15] <-----> رفع انحراف نوار انجام
[383, 29, 57, 384, 290, 400, 470, 32, 12] <-----> الارم تجهیز دلیل کاهش فشار پمپ گریس اتفاق افتاده_است
[499, 2, 239, 226, 32, 12] <-----> پایپهای جنوب چکش ثابت افتاده_است
[34, 16, 43, 15] <-----> رفع انحراف نوار انجام
[10, 78, 6, 128, 367, 151, 320, 343] <-----> نصب اسکافلد جهت تعمیر پرژکتور‌های جلو پولپیت بالینگ هماهنگ اقای ثقفی
[471, 89] <-----> گیربکس‌های روغن
[180, 153, 32, 12] <-----> پیچهای النی افتاده_است
[55, 77, 98] <-----> بازوی بالایی امده


In [62]:
#calculating Maximum Len
max_len=0
for item in seq:
    if len(item)>max_len:
        max_len=len(item)
print(max_len)

42


In [63]:
#making list with equal len
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_docs = pad_sequences(seq, padding = 'pre', maxlen= max_len)
print(padded_docs)

[[  0   0   0 ... 302  69  12]
 [  0   0   0 ... 128   2  15]
 [  0   0   0 ...  16  43  15]
 ...
 [  0   0   0 ... 469  14  80]
 [  0   0   0 ... 469  14  80]
 [  0   0   0 ...   0  26  24]]


In [64]:
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
embedding_vector_features = 30
model = Sequential()
model.add(Embedding(max_words, embedding_vector_features, input_length=max_len))
model.add(LSTM(256))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 42, 30)            15000     
                                                                 
 lstm (LSTM)                 (None, 256)               293888    
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 309,145
Trainable params: 309,145
Non-trainable params: 0
_________________________________________________________________
None


In [65]:
model.fit(padded_docs,y,batch_size=128,epochs=10, validation_split=0.2)

Epoch 1/10
42/42 [==============================] - 6s 112ms/step - loss: -37.0412 - accuracy: 0.1744 - val_loss: -54.6367 - val_accuracy: 0.1865
Epoch 2/10
42/42 [==============================] - 4s 100ms/step - loss: -72.8134 - accuracy: 0.1723 - val_loss: -76.4184 - val_accuracy: 0.1865
Epoch 3/10
42/42 [==============================] - 4s 101ms/step - loss: -96.7255 - accuracy: 0.1723 - val_loss: -97.7424 - val_accuracy: 0.1865
Epoch 4/10
42/42 [==============================] - 4s 101ms/step - loss: -119.9336 - accuracy: 0.1723 - val_loss: -118.0566 - val_accuracy: 0.1865
Epoch 5/10
42/42 [==============================] - 4s 102ms/step - loss: -142.4309 - accuracy: 0.1723 - val_loss: -138.2271 - val_accuracy: 0.1865
Epoch 6/10
42/42 [==============================] - 4s 101ms/step - loss: -164.9408 - accuracy: 0.1723 - val_loss: -158.2702 - val_accuracy: 0.1865
Epoch 7/10
42/42 [==============================] - 4s 101ms/step - loss: -187.4467 - accuracy: 0.1723 - val_loss: -17

In [76]:
tkn.texts_to_sequences(['شفتهای'])
#model.predict(['شکستن شفت'])

[[]]

In [42]:
temp['sentens']

0                     بازوهای بیرونی شماره گریسکاری نمیشود
1                                        نازل‌های اب تجهیز
2                                              تجهیز الارم
3                       دستگاه گلس بید عملکرد‌های کار فعال
4                            دستگاه گلاس بید استارت نمیشود
                               ...                        
59388                      رفتن دمای یاتاقان پنیون بال میل
59389                        نصب داربست اطراف بدنه بال میل
59390                                               شل بلت
59391    درخواست ساخت تعویض داکت‌های ماقبل نازل اسپری ا...
59392       دیسکانکت وکانکت سنسور هیدروکوپلینگ شات دان گرم
Name: sentens, Length: 59393, dtype: object

In [ ]:
df.iloc[:,1]

In [ ]:
temp['sentens'][0:15]

In [ ]:
sample=tf.transform(temp['sentens'][0:15]).toarray()

In [ ]:
sample

In [ ]:
model.predict(sample)

In [ ]:
revers = dict(zip(res.values(),res.keys()))

In [ ]:
revers[0]

In [ ]:
k=0
for a in model.predict(sample):
    
    print(df.iloc[:,1][k],'---->', revers[a])
    k +=1

In [ ]:
final_df.iloc[:,1].items

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer=CountVectorizer()
vocabulary=vectorizer.fit(final_df.iloc[:,1])

In [ ]:
X= vectorizer.transform(final_df.iloc[:,1])

In [ ]:
X.toarray()

In [ ]:
print(vocabulary.get_feature_names())

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(final_df.iloc[:,2])
#dictionary.save('/tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

In [ ]:
print(dictionary.token2id)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in final_df.iloc[:,2]]
corpora.MmCorpus.serialize('deerwester.mm', corpus)  # store to disk, for later use
print(corpus)

In [ ]:
from gensim import models

# train the model
tfidf = models.TfidfModel(corpus)

In [ ]:
print(tfidf[dictionary.doc2bow(['سایش','خوردگی'])])

In [ ]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

In [ ]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

In [ ]:
corpus_lsi

In [ ]:
lsi_model.print_topics(2)

In [ ]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc, as_text in zip(corpus_lsi, final_df.iloc[:,2]):
    print(doc, as_text)